In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'shedulers.csv', encoding='cp1251', sep=';')
df.head(10)

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


In [3]:
# оказывается, это весь файл
df.shape

(8, 4)

In [4]:
# смотрим максимальную длину, какой минимум должен быть для переменной в таблице
df['Расписание'].apply(lambda x: len(x)).max()

8

В таблице находятся значения 9999 для года, 
Сначала я думал их убрать, но в принципе это можно расценивать как "неограниченность в расисании"
Например, с 15.01.2019 0:00 Поставщик 2 работает по расписанию "ннддвсв"

In [5]:
# Данные в расписании указаны в неверном для MySQL формате
# Нужно поменять местами месяц и день, кроме того время не несет информации - его можно убрать
df['Дата начала расписания'] 

0    01.01.2019 0:00
1    11.01.2019 0:00
2    16.01.2019 0:00
3    01.01.2019 0:00
4    08.01.2019 0:00
5    15.01.2019 0:00
6    01.01.2019 0:00
7    02.02.2019 0:00
Name: Дата начала расписания, dtype: object

## Работа с SQL 

In [6]:
!pip3 install mysql-connector-python

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [7]:
import mysql.connector as mysql
from mysql.connector import Error

In [9]:
# Создаем подключение к локальному серверу и создаем базу данных
conn = mysql.connect(host='localhost', user='root', password='password')
cursor = conn.cursor()
#cursor.execute("CREATE DATABASE work")

In [10]:
# Кроме того, можно создать функцию для подключения к базе данных на сервере, с обработкой ошибок
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [11]:
#датасет называется work
conn = create_connection('localhost','root','password', 'work')

Connection to MySQL DB successful


In [12]:
# Функция для исполнения запросов
def execute_query(connection, query):
     cursor = connection.cursor()
     try:
         cursor.execute(query)
         connection.commit()
         print("Query executed successfully")
     except Error as e:
         print(f"The error '{e}' occurred")

In [16]:
# Создание таблицы (Ключ по ID_NAME, DATE_BEGIN)
# NAME и ID_NAME, по идее, несут один смысл и находятся в функциональной зависимости друг от друга
# Можно сделать отдельную таблицу [ID_NAME,NAME], чтобы этого избежать. Но по заданию, как я понял, оба параметра должны присутсвовать.

create_contractor_table = """
CREATE TABLE IF NOT EXISTS T_CONTRACTOR_SHERULER (
  ID_NAME INT NOT NULL, 
  NAME VARCHAR(20) NOT NULL, 
  SCHEDULE VARCHAR(10) NOT NULL, 
  DATE_BEGIN DATE NOT NULL, 
  DATE_END DATE NOT NULL, 
  CHECK (DATE_BEGIN <= DATE_END),
  PRIMARY KEY (NAME, DATE_BEGIN)
)
"""

In [14]:
# создаем таблицу
execute_query(conn, create_contractor_table)

Query executed successfully


In [ ]:
# заполение таблицы
cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO T_CONTRACTOR_SHERULER (ID_NAME, NAME, SCHEDULE, DATE_BEGIN,DATE_END)
                VALUES (%s,%s,%s,STR_TO_DATE(%s, '%d-%m-%Y') ,STR_TO_DATE(%s,'%d-%m-%Y'))
                ''',
                (row[1][-1],row[1],row[2],row[3][:10].replace('.','-'),row[4][:10].replace('.','-'))
                )

conn.commit()

In [130]:
# создание таблицы для рабочих смен, делаем проверку по дате
# FK на NAME нет смысла добавлять, потому что это не PK
# [NAME, DATE_BEGIN] - тоже не подходит для FK 
create_workday_table = """
CREATE TABLE IF NOT EXISTS T_CONTRACTOR_WORK_DAY (
  ID INT PRIMARY KEY AUTO_INCREMENT, 
  NAME VARCHAR(20) NOT NULL,
  DATE_BEGIN DATETIME NOT NULL, 
  DATE_END DATETIME NOT NULL, 
  CHECK (DATE_BEGIN <= DATE_END)
)
"""

In [131]:
# создаем таблицу
execute_query(conn, create_contractor_table)

Query executed successfully
